In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [3]:
llm.invoke("How will the weather be in munich today?")

AIMessage(content="I'm sorry, I am not able to provide real-time weather updates. I recommend checking a reliable weather forecasting website or app for the most up-to-date information on the weather in Munich today.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 17, 'total_tokens': 57, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0e508378-784c-4079-bb17-42b45c90589f-0', usage_metadata={'input_tokens': 17, 'output_tokens': 40, 'total_tokens': 57, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [7]:
from langchain_core.tools import tool


@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["munich"]:
        return "It's 15 degrees Celsius and cloudy."
    else:
        return "It's 32 degrees Celsius and sunny."


@tool
def check_seating_availability(location: str, seating_type: str):
    """Call to check seating availability."""
    if location.lower() == "munich" and seating_type.lower() == "outdoor":
        return "Yes, we still have seats available outdoors."
    elif location.lower() == "munich" and seating_type.lower() == "indoor":
        return "Yes, we have indoor seating available."
    else:
        return "Sorry, seating information for this location is unavailable."


tools = [get_weather, check_seating_availability]

In [5]:
llm_with_tools = llm.bind_tools(tools)

In [6]:
result = llm_with_tools.invoke("How will the weather be in munich today?")
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_h6yjQp64Y6iNd3rpCZnMwLVo', 'function': {'arguments': '{"location":"Munich"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 84, 'total_tokens': 101, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-64b4d902-53f6-4262-940b-0bc55c9217df-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Munich'}, 'id': 'call_h6yjQp64Y6iNd3rpCZnMwLVo', 'type': 'tool_call'}], usage_metadata={'input_tokens': 84, 'output_tokens': 17, 'total_tokens': 101, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
result.tool_calls

In [ ]:
result = llm_with_tools.invoke(
    "How will the weather be in munich today? Do you still have seats outdoor available?"
)
result

In [ ]:
result.tool_calls

In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [
    HumanMessage(
        "How will the weather be in munich today? Do you still have seats outdoor available?"
    )
]
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

In [ ]:
messages

In [ ]:
tool_mapping = {
    "get_weather": get_weather,
    "check_seating_availability": check_seating_availability,
}

In [ ]:
llm_output.tool_calls

In [ ]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [ ]:
messages

In [ ]:
llm_with_tools.invoke(messages)